In [1]:
import open3d as o3d
import numpy as np
import pathlib
import copy
import cv2
import time
from IPython.display import display, clear_output
from scipy.spatial.transform import Rotation as R
from typing import Generator

In [2]:
# helper function to load each image, dmap, etc..
def dataloader_dmap(dataset_path: str, ground_truth: bool = False):
    """_summary_

    Args:
        dataset_path (str): _description_
        ground_truth (bool, optional): _description_. Defaults to False.

    Yields:
        _type_: _description_
    """
    counter = 0
    while True:
        try:
            render_path = dataset_path + f'/render{counter}'
            if ground_truth:
                dmap_path = render_path + '/dmap.npy'
            else:
                dmap_path = render_path + '/nndepth.npy'
            wrench_path = render_path + '/image8.png'
            dmap = np.load(dmap_path)                           # Depth maps are in millimeters
            wrench_img = cv2.imread(wrench_path)
            counter +=1
            yield dmap, wrench_img
        except:
            break
        
def dataloader_ply(dataset_path: str):
    counter = 0
    while True:
        try:
            render_path = dataset_path + f'/render{counter}'
            ply_path = render_path + f'/{counter}.ply'
            ply = o3d.io.read_point_cloud(ply_path)
            counter +=1
            yield ply
        except:
            break
    
        
class Stitcher():
    def __init__(self, voxel_size=0.2, radius_normal_weight=2.0, radius_fpfh_feature_weight=5.0, max_nn_fpfh_feature=230, 
                 max_nn_normal=230, max_nn_local_reg=100, dist_correspond_checker_weight=1.5, edge_length_correspond_checker=0.95,
                 ransac_max_iterations=10000000, ransac_confidence=0.999, max_correspond_dist=0.2):
        """_summary_

        Args:
            voxel_size (float, optional): _description_. Defaults to 0.2.
            radius_normal_weight (float, optional): _description_. Defaults to 2.0.
            radius_fpfh_feature_weight (float, optional): _description_. Defaults to 5.0.
            max_nn_fpfh_feature (int, optional): _description_. Defaults to 230.
            max_nn_normal (int, optional): _description_. Defaults to 230.
            max_nn_local_reg (int, optional): _description_. Defaults to 100.
            dist_correspond_checker_weight (float, optional): _description_. Defaults to 1.5.
            edge_length_correspond_checker (float, optional): _description_. Defaults to 0.95.
            ransac_max_iterations (int, optional): _description_. Defaults to 10000000.
            ransac_confidence (float, optional): _description_. Defaults to 0.999.
            max_correspond_dist (float, optional): _description_. Defaults to 0.2.
        """
        self.voxel_size = voxel_size
        self.max_nn_normal = max_nn_normal
        self.radius_normal =  voxel_size * radius_normal_weight
        self.radius_fpfh_feature = voxel_size * radius_fpfh_feature_weight
        self.max_nn_fpfh_feature = max_nn_fpfh_feature
        self.dist_correspond_checker = voxel_size * dist_correspond_checker_weight      # G
        self.edge_length_correspond_checker = edge_length_correspond_checker            # G
        self.ransac_max_iterations = ransac_max_iterations                              # G
        self.ransac_confidence = ransac_confidence                                      # G
        self.max_correspond_dist = max_correspond_dist
        self.radius_local_reg = voxel_size * radius_normal_weight
        self.max_nn_local_reg = max_nn_local_reg
        self.current_transform = np.eye(4)                                              # Identity Matrix
        self.transforms_history = [self.current_transform]                              # This will store all transforms/poses of each pcd
                                                                                        # is really helpfull to evaluate metrics and comparisons.
    
    @classmethod
    def generate_pointcloud(cls, img: np.array, dmap: np.array, pcd: o3d.cpu.pybind.geometry.PointCloud) -> None:
        """
        Populates point cloud given the information provided by the image and depth map.

        Args:
            img (np.array): PNG image
            dmap (np.array): Depth map/image
            pcd (o3d.cpu.pybind.geometry.PointCloud): Point cloud object
        """
        height, width = dmap.shape
        xyz = np.ones(shape=(height, width, 3))  
        x = np.arange(width).reshape((1, width))
        y = np.arange(height).reshape((height, 1))
        xyz[:, :, 0] = xyz[:, :, 0]*x                                 # Fill with horizontal indexes
        xyz[:, :, 1] = xyz[:, :, 1]*y                                 # Fill with vertical indexes  
        xyz[:, :, 2] = dmap                                           # Fill with depth, z coords w.r.t camera frame  
        xyz[:, :, 0] = 0.1655 * (xyz[:, :, 0]-80) * xyz[:, :, 2]/80   # Updated with x coords w.r.t camera frame 
        xyz[:, :, 1] = 0.1655 * (xyz[:, :, 1]-80) * xyz[:, :, 2]/80   # Updated with y coords w.r.t camera frame 
        xyz = xyz.reshape(width*height, -1)                           # Shape required by open3d
        rgb = img.reshape((width*height, -1))[::-1]
        # *********************************************************
        # pcd is a mutable object so no need to return the updated 
        # object as modifications are applied to the original
        pcd.points = o3d.utility.Vector3dVector(xyz)
        pcd.colors = o3d.utility.Vector3dVector(rgb)
    
    
    @classmethod
    def merge_point_clouds(cls, source: o3d.cpu.pybind.geometry.PointCloud, target: o3d.cpu.pybind.geometry.PointCloud, transform: np.array) -> None:
        """
        Transform source point cloud to target point cloud reference frame and merges it in target point cloud.
        *Changes are reflected in target object itself as this behvaious is same as being passed by reference in c++ thus no need to return anything.

        Args:
            source (o3d.cpu.pybind.geometry.PointCloud): Point cloud to be transformed.
            target (o3d.cpu.pybind.geometry.PointCloud): Point cloud that will be merged with the transformed source point cloud.
            transform (np.array):                        Homogeneous transformation matrix of shape 4x4. This matrix MUST represent the transform form source to target
        """
        source.transform(transform)
        target += source
        # Is not necessary to return source since changes are applied directly to the object 
        
        
    def point_cloud_start_location(self, pcd: o3d.cpu.pybind.geometry.PointCloud, transform: np.array) -> None:
        """
        Transforms both point clouds (source & target) to the same initial location/pose given by the transform matrix.
        *Changes are reflected in source and target object itself as this behvaious is same as being passed by reference in c++ thus no need to return anything.

        Args:
            pcd (o3d.cpu.pybind.geometry.PointCloud): Point cloud to be transformed
            target (o3d.cpu.pybind.geometry.PointCloud): Point cloud to be transformed
            transform (np.array):                        Homogeneous transformation matrix shape 4x4. This matrix represent the starting pose/location for both point clouds
        """
        pcd.transform(transform)
        
         
    #def point_clouds_start_location(self, source: o3d.cpu.pybind.geometry.PointCloud, target: o3d.cpu.pybind.geometry.PointCloud, transform: np.array) -> None:
    #    """
    #    Transforms both point clouds (source & target) to the same initial location/pose given by the transform matrix.
    #    *Changes are reflected in source and target object itself as this behvaious is same as being passed by reference in c++ thus no need to return anything.
    #
    #    Args:
    #        source (o3d.cpu.pybind.geometry.PointCloud): Point cloud to be transformed
    #        target (o3d.cpu.pybind.geometry.PointCloud): Point cloud to be transformed
    #        transform (np.array):                        Homogeneous transformation matrix shape 4x4. This matrix represent the starting pose/location for both point clouds
    #    """
    #    source.transform(transform)
    #    target.transform(transform) 
        
        
    def global_and_icp_registration_v2(self, source: o3d.cpu.pybind.geometry.PointCloud, target: o3d.cpu.pybind.geometry.PointCloud) -> o3d.cpu.pybind.pipelines.registration.RegistrationResult:
        """
        Calculates the results necessary for transforming source point cloud to target point cloud as best as possible. 

        Args:
            source (o3d.cpu.pybind.geometry.PointCloud): Point cloud that is looking to be transformed
            target (o3d.cpu.pybind.geometry.PointCloud): Point Cloud that works as the reference base

        Returns:
            o3d.cpu.pybind.pipelines.registration.RegistrationResult: Object that contains registration results such as transformation, correspondence_set, fitness, inlier_rmse
        """
        source_down, source_fpfh = self._preprocess_point_cloud(source)
        target_down, target_fpfh = self._preprocess_point_cloud(target)
        result_ransac = self._global_registration(source_down, target_down, source_fpfh, target_fpfh)
        result_icp = self._local_registration(source, target, result_ransac.transformation)
        return result_icp
    
    
    def set_current_transform(self, transform: np.array) -> None:
        """
        Updates current transform. The input MUST represent the transform of the newest point cloud to the world frame.
        E.g.  current_transform = stitcher.current_transform @ result_icp.transformation  

        Args:
            transform (np.array): Homogeneous transformation matrix shape 4x4. 
        """
        self.current_transform = transform
    


    def _preprocess_point_cloud(self, pcd: o3d.cpu.pybind.geometry.PointCloud) -> tuple[o3d.cpu.pybind.geometry.PointCloud, o3d.cpu.pybind.pipelines.registration.Feature]:
        """
        Creates a simplified point cloud (downsampled) of the input point cloud.

        Args:
            pcd (o3d.cpu.pybind.geometry.PointCloud): Point cloud

        Returns:
            tuple[o3d.cpu.pybind.geometry.PointCloud, 
            o3d.cpu.pybind.pipelines.registration.Feature]: Tuple comtaining simplified point cloud  and its corresponding features descriptor.
        """
        pcd_down = pcd.voxel_down_sample(self.voxel_size)
        pcd_down = self.outlier_removal(pcd_down)
        pcd_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=self.radius_normal, max_nn=self.max_nn_normal))
        pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(pcd_down, o3d.geometry.KDTreeSearchParamHybrid(radius=self.radius_fpfh_feature, max_nn=self.max_nn_fpfh_feature))
        return pcd_down, pcd_fpfh


    def outlier_removal(self, pcd):
        #pcd, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
        pcd, ind = pcd.remove_radius_outlier(nb_points=25, radius=0.5)
        return pcd
    
    def _global_registration(self, source_down: o3d.cpu.pybind.geometry.PointCloud, 
                             target_down: o3d.cpu.pybind.geometry.PointCloud, 
                             source_fpfh: o3d.cpu.pybind.pipelines.registration.Feature, 
                             target_fpfh:o3d.cpu.pybind.pipelines.registration.Feature) -> o3d.cpu.pybind.pipelines.registration.RegistrationResult:
        """
        Calculates the results necessary for transforming source point cloud to target point cloud as a best initial allignment.
        Point clouds can be placed anywehere, global registration will produce the a rough best alignment. This requires fine tunning/local registration
        to get better allignment. 

        Args:
            source_down (o3d.cpu.pybind.geometry.PointCloud): Point cloud that is looking to be transformed.
            target_down (o3d.cpu.pybind.geometry.PointCloud): Point cloud that works as the reference base.
            source_fpfh (o3d.cpu.pybind.pipelines.registration.Feature): Point cloud corresponding features descriptor.
            target_fpfh (o3d.cpu.pybind.pipelines.registration.Feature): Point cloud corresponding features descriptor.

        Returns:
            o3d.cpu.pybind.pipelines.registration.RegistrationResult: Object that contains registration results such as transformation, correspondence_set, fitness, inlier_rmse
        """
        result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
            source_down, target_down, source_fpfh, target_fpfh, True, self.dist_correspond_checker,
            o3d.pipelines.registration.TransformationEstimationPointToPoint(False), 3, 
            [
                o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(self.edge_length_correspond_checker),
                o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(self.dist_correspond_checker)
            ], 
            o3d.pipelines.registration.RANSACConvergenceCriteria(self.ransac_max_iterations, self.ransac_confidence))
        return result


    def _local_registration(self, source: o3d.cpu.pybind.geometry.PointCloud, target: o3d.cpu.pybind.geometry.PointCloud, transform: np.array) -> o3d.cpu.pybind.pipelines.registration.RegistrationResult:
        """
        Calculates the results necessary for transforming source point cloud to target point cloud as best as possible.
        Point clouds need to be placed together on the area they are meant to be matched. As this method creates only fine tunning allignment. 

        Args:
            source (o3d.cpu.pybind.geometry.PointCloud): Point cloud that is looking to be transformed.
            target (o3d.cpu.pybind.geometry.PointCloud): Point cloud that works as the reference base.
            transform (np.array):                        Homogeneous transformation matrix shape 4x4. This matrix represents the global alignment between the point clouds (source -> target)

        Returns:
            o3d.cpu.pybind.pipelines.registration.RegistrationResult: Object that contains registration results such as transformation, correspondence_set, fitness, inlier_rmse
        """
        source.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=self.radius_local_reg, max_nn=self.max_nn_local_reg))
        target.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=self.radius_local_reg, max_nn=self.max_nn_local_reg))
        result = o3d.pipelines.registration.registration_icp(source, target, self.max_correspond_dist, transform, o3d.pipelines.registration.TransformationEstimationPointToPlane())
        #result = o3d.pipelines.registration.registration_colored_icp(source, target, voxel_size, result.transformation)
        return result


    #def global_and_icp_registration(self, source: o3d.cpu.pybind.geometry.PointCloud, target: o3d.cpu.pybind.geometry.PointCloud) -> np.array:
        #    """
        #    Calculates the transformation matrix necessary for transforming source point cloud to target point cloud as best as possible.
        #
        #    Args:
        #        source (o3d.cpu.pybind.geometry.PointCloud): Point cloud that is looking to be transformed
        #        target (o3d.cpu.pybind.geometry.PointCloud): Point Cloud that works as a reference base
        #
        #    Returns:
        #        np.array: Homogeneous transformation matrix shape 4x4. This matrix represents the transform form source to target
        #    """
        #    self._set_point_clouds_init_transform(source, target)
        #    source_down, target_down, source_fpfh, target_fpfh = self._simplify_point_clouds(source, target)
        #    result_ransac = self._global_registration(source_down, target_down, source_fpfh, target_fpfh)
        #    result_icp = self._local_registration(source, target, result_ransac.transformation)
        #    self.current_transform = self.current_transform @ result_icp.transformation
        #    self.merge_point_clouds(source, target, result_icp.transformation)
        #    # At this point source contains the merge between both point clouds
        #    # Is not necessary to return source since changes are applied directly to the object
        #    return result_icp.transformation
        #
        #
        #def _set_point_clouds_init_transform(self, source: o3d.cpu.pybind.geometry.PointCloud, target: o3d.cpu.pybind.geometry.PointCloud) -> None:
        #    target.transform(self.current_transform)
        #    source.transform(self.current_transform)
        
        
        #def _simplify_point_clouds(self, source: o3d.cpu.pybind.geometry.PointCloud, target: o3d.cpu.pybind.geometry.PointCloud) -> tuple[o3d.cpu.pybind.geometry.PointCloud,
        #                                                                                                                                  o3d.cpu.pybind.geometry.PointCloud,
        #                                                                                                                                  o3d.cpu.pybind.pipelines.registration.Feature,
        #                                                                                                                                  o3d.cpu.pybind.pipelines.registration.Feature]:
        #    """_summary_
        #
        #    Args:
        #        source (o3d.cpu.pybind.geometry.PointCloud): _description_
        #        target (o3d.cpu.pybind.geometry.PointCloud): _description_
        #
        #    Returns:
        #        tuple[o3d.cpu.pybind.geometry.PointCloud, 
        #              o3d.cpu.pybind.geometry.PointCloud, 
        #              o3d.cpu.pybind.pipelines.registration.Feature, 
        #              o3d.cpu.pybind.pipelines.registration.Feature]: Tuple containing simplified source and target pointclouds and their corresponding features descriptors. 
        #    """
        #    source_down, source_fpfh = self._preprocess_point_cloud(source)
        #    target_down, target_fpfh = self._preprocess_point_cloud(target)
        #    #source_down, outlier_index = source_down.remove_radius_outlier(nb_points=25,radius=0.5)
        #    #source_down, outlier_index = target_down.remove_radius_outlier(nb_points=25,radius=0.5)
        #    return source_down, target_down, source_fpfh, target_fpfh


def get_transform(img_query, img_target):
    orb = cv2.ORB_create()
    matcher = cv2.BFMatcher_create(cv2.NORM_HAMMING, crossCheck=True)
    kpts_query,  desc_query = orb.detectAndCompute(img_query, None)
    kpts_target, desc_target = orb.detectAndCompute(img_target, None)
    matches = matcher.match(desc_query, desc_target)
    kpts_query_filtered = []
    kpts_target_filtered = []
    for match in matches:
        kpts_query_filtered = kpts_query[match.queryIdx]
        kpts_target_filtered = kpts_target[match.trainIdx]
    fund_mtx = cv2.findFundamentalMat(kpts_query_filtered, 
                                      kpts_target_filtered, 
                                      cv2.FM_RANSAC, 3, 0.99)
    

In [3]:
#  *** **************** ******************** ******************** ******************** ******************** ***
#  ******************** This approach is for stitching NEW point cloud with PREVIOUS. ******************** 
#  *** **************** ******************** ******************** ******************** ******************** ***

root_path = str(pathlib.Path('').parent.resolve().parent)

# Select one dataset
dataset = 'LA200'
dataset_path = root_path + f'/datasets/{dataset}'

# Represents how many individual point clouds are going to be grouped
rolling_window = 5
pcd_size = 128*128
rolling_window_total_points = rolling_window * pcd_size
    
stitcher = Stitcher()
dataset = dataloader(dataset_path, ground_truth=True)
point_cloud_0 = o3d.geometry.PointCloud()
point_cloud_1 = o3d.geometry.PointCloud()

dmap_0, img_0 = next(dataset)
stitcher.generate_pointcloud(img_0, dmap_0, point_cloud_0)
point_cloud_0 = stitcher.outlier_removal(point_cloud_0)

history_point_cloud = history_point_cloud_down = copy.deepcopy(point_cloud_0)
history_points_xyz = np.array(history_point_cloud.points) 

vis = o3d.visualization.Visualizer()
vis.create_window(height=1000, width=1000)
vis.add_geometry(history_point_cloud_down)

print("Image: 0")
for i, package in enumerate(dataset):
    dmap_1, img_1 = package
    stitcher.generate_pointcloud(img_1, dmap_1, point_cloud_1)
    point_cloud_1 = stitcher.outlier_removal(point_cloud_1)
    
    point_cloud_1_temp = copy.deepcopy(point_cloud_1)
    
    if len(history_points_xyz) >= (rolling_window_total_points):
        pass
    else:
        stitcher.point_cloud_start_location(point_cloud_0, stitcher.current_transform)
        
    stitcher.point_cloud_start_location(point_cloud_1, stitcher.current_transform)                     # As the name implies first initialize both pcls on specified loc.
    result_icp = stitcher.global_and_icp_registration_v2(point_cloud_1, point_cloud_0)                 # Calculate the best transform starting between both point clouds
    current_transform = stitcher.current_transform @ result_icp.transformation                         # Calculate the new current transform of the trayectory. 
    stitcher.set_current_transform(current_transform)                                                  # This is useful for the next CONSECUTIVE pcl. As the starting loc.
    
    history_point_cloud += point_cloud_1.transform(result_icp.transformation)
    
    vis.update_geometry(history_point_cloud_down)
    vis.poll_events()
    vis.update_renderer()
    
    history_points_xyz = np.array(history_point_cloud.points)
    
    if len(history_points_xyz) >= (rolling_window_total_points): 
        point_cloud_0 = o3d.geometry.PointCloud()
        point_cloud_0.points = o3d.utility.Vector3dVector(history_points_xyz[-(rolling_window_total_points):, :])
    else:
        point_cloud_0 = copy.deepcopy(point_cloud_1_temp)

    print(f"Image: {i+1}")
    if (i+1) == 199:
        break

vis.destroy_window()
o3d.visualization.draw_geometries([history_point_cloud])

Image: 0
Image: 1
Image: 2
Image: 3
Image: 4
Image: 5
Image: 6
Image: 7
Image: 8
Image: 9
Image: 10
Image: 11
Image: 12
Image: 13
Image: 14
Image: 15
Image: 16
Image: 17
Image: 18
Image: 19
Image: 20
Image: 21
Image: 22
Image: 23
Image: 24
Image: 25
Image: 26
Image: 27
Image: 28
Image: 29
Image: 30
Image: 31
Image: 32
Image: 33
Image: 34
Image: 35
Image: 36
Image: 37
Image: 38
Image: 39
Image: 40
Image: 41
Image: 42
Image: 43
Image: 44
Image: 45
Image: 46
Image: 47
Image: 48
Image: 49
Image: 50
Image: 51
Image: 52
Image: 53
Image: 54
Image: 55
Image: 56
Image: 57
Image: 58
Image: 59
Image: 60
Image: 61
Image: 62
Image: 63
Image: 64
Image: 65
Image: 66
Image: 67
Image: 68
Image: 69
Image: 70
Image: 71
Image: 72
Image: 73
Image: 74
Image: 75
Image: 76
Image: 77
Image: 78
Image: 79
Image: 80
Image: 81
Image: 82
Image: 83
Image: 84
Image: 85
Image: 86
Image: 87
Image: 88
Image: 89
Image: 90
Image: 91
Image: 92
Image: 93
Image: 94
Image: 95
Image: 96
Image: 97
Image: 98
Image: 99
Image: 100

In [6]:
dmap_0.shape

(160, 160)

In [3]:
#  *** **************** ******************** ******************** ******************** ******************** ***
#  ******************** V2-This approach is for stitching NEW point cloud with PREVIOUS. ******************** 
#  *** **************** ******************** ******************** ******************** ******************** ***

root_path = str(pathlib.Path('').parent.resolve().parent)

# Select one dataset
dataset = 'LA200_inf'
dataset_path = root_path + f'/datasets/{dataset}'

# Represents how many individual point clouds are going to be grouped
rolling_window = 10
pcd_size = 128*128
rolling_window_total_points = rolling_window * pcd_size
    
stitcher = Stitcher()
dataset = dataloader_ply(dataset_path)
point_cloud_0 = o3d.geometry.PointCloud()
point_cloud_1 = o3d.geometry.PointCloud()

point_cloud_0 = next(dataset)
point_cloud_0 = stitcher.outlier_removal(point_cloud_0)
#stitcher.generate_pointcloud(img_0, dmap_0, point_cloud_0)

history_point_cloud = copy.deepcopy(point_cloud_0)
history_points_xyz = np.array(history_point_cloud.points) 

#o3d.visualization.draw_geometries([point_cloud_0])

vis = o3d.visualization.Visualizer()
vis.create_window(height=1000, width=1000)
vis.add_geometry(history_point_cloud)


for i, package in enumerate(dataset):
    point_cloud_1 = package
    point_cloud_1 = stitcher.outlier_removal(point_cloud_1)
    point_cloud_1_temp = copy.deepcopy(point_cloud_1)
    
    if len(history_points_xyz) >= (rolling_window_total_points):
        pass
    else:
        stitcher.point_cloud_start_location(point_cloud_0, stitcher.current_transform)
        
    stitcher.point_cloud_start_location(point_cloud_1, stitcher.current_transform)                     # As the name implies first initialize both pcls on specified loc.
    result_icp = stitcher.global_and_icp_registration_v2(point_cloud_1, point_cloud_0)                 # Calculate the best transform starting between both point clouds
    current_transform = stitcher.current_transform @ result_icp.transformation                         # Calculate the new current transform of the trayectory. 
    stitcher.set_current_transform(current_transform)                                                  # This is useful for the next CONSECUTIVE pcl. As the starting loc.
    
    history_point_cloud += point_cloud_1.transform(result_icp.transformation)
    
    vis.update_geometry(history_point_cloud)
    vis.poll_events()
    vis.update_renderer()
    
    history_points_xyz = np.array(history_point_cloud.points)
    if len(history_points_xyz) >= (rolling_window_total_points): 
        point_cloud_0 = o3d.geometry.PointCloud()
        point_cloud_0.points = o3d.utility.Vector3dVector(history_points_xyz[-(rolling_window_total_points):, :])
    else:
        point_cloud_0 = copy.deepcopy(point_cloud_1_temp)
    
    #if (i+1)%50 == 0:
    #    o3d.visualization.draw_geometries([history_point_cloud])
    
    if (i+1) == 199:
        break
vis.destroy_window()



In [ ]:
o3d.visualization.draw_geometries([history_point_cloud])

In [ ]:
#  *** **************** ******************** ******************** ******************** ******************** ***
#  ******************** This approach is for stitching all point clouds one after another. ******************** 
#  *** **************** ******************** ******************** ******************** ******************** ***

root_path = str(pathlib.Path('').parent.resolve().parent)

# Select one dataset
dataset = 'LA10'
dataset_path = root_path + f'/datasets/{dataset}'

# Represents how many individual point clouds are going to be grouped
group_size = 5  
stitcher = Stitcher()
dataset = dataloader(dataset_path)

point_cloud_0 = o3d.geometry.PointCloud()
point_cloud_1 = o3d.geometry.PointCloud()
history_point_cloud = copy.deepcopy(point_cloud_0)

dmap_0, img_0 = next(dataset)
stitcher.generate_pointcloud(img_0, dmap_0, point_cloud_0) 
#stitcher.transform_2_world_frame(point_cloud_0)

for i, package in enumerate(dataset):
    dmap_1, img_1 = package
    stitcher.generate_pointcloud(img_1, dmap_1, point_cloud_1)
    
    point_cloud_1_temp = copy.deepcopy(point_cloud_1)
    print(f"PCL1_TEMP: {np.array(point_cloud_1_temp.points)[0,:]}")
    
    _ = stitcher.global_and_icp_registration(point_cloud_1, point_cloud_0)
    print(f"PCL1_FINAL: {np.array(point_cloud_1.points)[0,:]}")
    merged_point_clouds = copy.deepcopy(point_cloud_1)
    history_point_cloud += merged_point_clouds
    
    
    # The next starting point_cloud_0point is going to be the previous point cloud
    point_cloud_0 = copy.deepcopy(point_cloud_1_temp)
    
    if (i+1)%10 == 0:
        #point_cloud_0.paint_uniform_color([1, 0, 0])
        #point_cloud_1.paint_uniform_color([0, 1, 0])
        o3d.visualization.draw_geometries([history_point_cloud])
    if (i+1) == 1:
        break
    print("Iteration: ",i)
    
# Final view of the stitched object    
#point_cloud_0, _ = point_cloud_0.remove_radius_outlier(nb_points=25, radius=0.5)

#o3d.visualization.draw_geometries([point_cloud_0])

In [ ]:
a = np.array([[-5.74930286e-05,  9.99999993e-01,  1.02630216e-04,  1.32672791e-03],
              [-7.75953563e-05, -1.02634677e-04,  9.99999992e-01,  1.28570857e-03],
              [ 9.99999995e-01,  5.74850645e-05,  7.76012565e-05, -4.10673829e-03],
              [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

b = np.array([[ 6.98218409e-05,  9.99999997e-01, -2.49201977e-05, -8.26632295e-03],
              [-2.30308338e-07,  2.49202138e-05,  1.00000000e+00,  3.30637134e-02],
              [ 9.99999998e-01, -6.98218352e-05,  2.32048313e-07, -4.69649984e-05],
              [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

trans_init = np.eye(4)
        


root_path = str(pathlib.Path('').parent.resolve().parent)
# Select one dataset
dataset = 'LA10'
dataset_path = root_path + f'/datasets/{dataset}'
dataset = dataloader(dataset_path)
stitcher = Stitcher()

point_cloud_0 = o3d.geometry.PointCloud()
point_cloud_1 = o3d.geometry.PointCloud()

dmap_0, img_0 = next(dataset)
stitcher.generate_pointcloud(img_0, dmap_0, point_cloud_0) 

dmap_1, img_1 = next(dataset)
stitcher.generate_pointcloud(img_1, dmap_1, point_cloud_1) 

point_cloud_0.paint_uniform_color([0, 0, 1])
point_cloud_1.paint_uniform_color([1, 0, 0])

##point_cloud_1 = point_cloud_1.transform(a)
##point_cloud_0.transform(trans_init)
#point_cloud_0.transform(b @ trans_init)
#point_cloud_0 += point_cloud_1

point_cloud_0.transform(trans_init)

world_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])
pcd_0_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=trans_init[:3, -1])
pcd_0_frame.rotate(trans_init[:3, :3])

o3d.visualization.draw_geometries([point_cloud_0, point_cloud_1, pcd_0_frame])



In [ ]:
import time

In [ ]:
#  *** **************** ******************** ******************** ******************** ******************** ***
#  ******************** This approach is in DEVELOPMENT. ******************** 
#  *** **************** ******************** ******************** ******************** ******************** ***

root_path = str(pathlib.Path('').parent.resolve().parent)

# Select one dataset
dataset = 'LA10'
dataset_path = root_path + f'/datasets/{dataset}'

# Represents how many individual point clouds are going to be grouped
group_size = 5  
stitcher = Stitcher()
dataset = dataloader(dataset_path)

point_cloud_0 = o3d.geometry.PointCloud()
point_cloud_1 = o3d.geometry.PointCloud()

dmap_0, img_0 = next(dataset)
stitcher.generate_pointcloud(img_0, dmap_0, point_cloud_0) 

current_transform = stitcher.init_transform
whole_point_cloud = copy.deepcopy(point_cloud_0)

for i, package in enumerate(dataset):
    
    dmap_1, img_1 = package
    stitcher.generate_pointcloud(img_1, dmap_1, point_cloud_1)
    
    # Original, Before any transformation
    point_cloud_1_temp = copy.deepcopy(point_cloud_1)    

    # From current image to previous image
    # Once the following line is executed point_cloud_1 will be transformed but we returned the transformed applied
    icp_transform = stitcher.global_and_icp_registration(point_cloud_0, point_cloud_1)
    
    point_cloud_0 = copy.deepcopy(point_cloud_1_temp)

    current_transform = current_transform @ icp_transform 
    point_cloud_1_temp = point_cloud_1_temp.transform(icp_transform)
    
    whole_point_cloud += point_cloud_1_temp
    point_cloud_1 = o3d.geometry.PointCloud()
    
    
    if (i+1)%1 == 0:
        print("dddd")
        o3d.visualization.draw_geometries([point_cloud_0])
      
    if (i+1) == 2:
        break
    print("Iteration: ",i)
    
    
    
# Final view of the stitched object    
#point_cloud_0, _ = point_cloud_0.remove_radius_outlier(nb_points=25, radius=0.5)
o3d.visualization.draw_geometries([whole_point_cloud])

In [ ]:
whole_point_cloud
current_transform

In [ ]:
o3d.visualization.draw_geometries([point_cloud_1])

In [ ]:
dataset = dataloader(dataset_path)

pc0 = o3d.geometry.PointCloud()
pc1 = o3d.geometry.PointCloud()


dmap0, img0 = next(dataset)
dmap1, img1 = next(dataset)

generate_pointcloud(img0, dmap0, pc0)
generate_pointcloud(img1, dmap1, pc1)


In [ ]:
%%timeit
generate_pointcloud(img0, dmap0, pc0)

In [ ]:
pc0, np.asarray(pc0.points)

In [ ]:
pc1, np.asarray(pc1.points)

In [ ]:
pc0 = pc1 + pc0

In [ ]:
pc0, np.asarray(pc0.points)

In [ ]:
pc1.points[0] = np.array([100.,100.,100.])

# DEBUG

In [ ]:
width = height = 5
xyz = np.ones(shape=(height, width, 3))  

x = np.arange(width).reshape((1, width))
y = np.arange(height).reshape((height, 1))


xyz[:, :, 0] = xyz[:, :, 0]*x   
xyz[:, :, 1] = xyz[:, :, 1]*y  


print(xyz[:,:,0])
print()
print(xyz[:,:,1])
print()
print(xyz[:,:,2])

xyz = xyz.reshape(width*height, -1)
print(xyz)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)

In [ ]:
# This cell is just for visualization purposes to understand how the dataset looks like
for i, package in enumerate(dataset):
    dmap, wrench_img = package
    print('{}, \t{}'.format(type(dmap), type(wrench_img)))
    plt.imshow(dmap)
    #cv2.imshow("danbots", wrench_img)
    plt.pause(0.1)
    plt.draw()
    clear_output(wait = True)


###
Currently the dataset consist on 3 loops. Each loop the camera is at a different angle, this means the first 100 picures correspond to one sequence, the next 100 corresponds to another sequence and so on...
The first approach given how the data is not continous as a single big trajectory, will be about computing the whole structure/point-cloud for each trajectory and then merging the 3 final point-clouds to get the end results  

In [ ]:
img0.reshape((160*160, -1))

In [ ]:
np.array(pc0.colors)

# Optimized function for Deployment

In [ ]:
from PIL import Image
def generate_pointcloud_v3(mask, rgb_file, mydepth, ply_file):
    img = Image.open(rgb_file)
    img = np.array(img)[:,:,:-1]
    
    height, width = mydepth.shape
    xyz = np.ones(shape=(height, width, 3))  
    
    x = np.arange(width).reshape((1, width))
    y = np.arange(height).reshape((height, 1))
    
    xyz[:, :, 0] = xyz[:, :, 0]*x                                 # Fill with horizontal indexes
    xyz[:, :, 1] = xyz[:, :, 1]*y                                 # Fill with vertical indexes  
    
    xyz[:, :, 2] = mydepth                                        # Fill with depth, z coords w.r.t camera frame  
    xyz[:, :, 0] = 0.1655 * (xyz[:, :, 0]-80) * xyz[:, :, 2]/80   # Updated with x coords w.r.t camera frame 
    xyz[:, :, 1] = 0.1655 * (xyz[:, :, 1]-80) * xyz[:, :, 2]/80   # Updated with y coords w.r.t camera frame 
    
    xyz = xyz.reshape(width*height, -1)                           # Shape required by open3d
    rgb = img.reshape((width*height, -1))
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)
    pcd.colors = o3d.utility.Vector3dVector(rgb)
    
    o3d.io.write_point_cloud(ply_file, pcd)


In [ ]:
demo_img = "/home/corcasta/Danbots/stitcher/datasets/LA10/render0/image0.png"

In [ ]:

%%timeit
generate_pointcloud_v3(None, demo_img, dmap0, "/home/corcasta/Danbots/stitcher/notebooks/data.ply")

In [ ]:
pcd = o3d.io.read_point_cloud("/home/corcasta/Danbots/stitcher/notebooks/data.ply")
o3d.visualization.draw_geometries([pcd])